In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import datetime as dt
from sklearn.cluster import KMeans
import random as ran
from sklearn.metrics.pairwise import euclidean_distances as euc
from functools import reduce

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

from src import crs
from IPython.display import clear_output, Markdown, display

In [3]:
# 1. load_excel
data_path = "data/apt_all.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

In [4]:
# 2. data preprocessing
p, m = crs.utils.data_preprocessing(xlsx)

m.set_index("month", inplace=True)
m.head()

/Users/formegusto/Desktop/idoformegusto/development/research/what-is-public-fee/src/crs/utils/data_preprocessing.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  datas_df[household_name] = household_df[col][3:].to_list()
/Users/formegusto/Desktop/idoformegusto/development/research/what-is-public-fee/src/crs/utils/data_preprocessing.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  month_usage_df[name] = [


,아파트1-101-1002,아파트1-101-104,아파트1-101-1102,아파트1-101-1202,아파트1-101-1302,아파트1-101-1401,아파트1-101-1403,아파트1-101-1502,아파트1-101-1504,아파트1-101-1602,...,아파트4-104-101,아파트4-104-1301,아파트4-104-1501,아파트4-104-301,아파트4-105-102,아파트4-105-103,아파트4-105-1102,아파트4-105-1104,아파트4-105-601,아파트4-105-803
month,,,,,,,,,,,,,,,,,,,,,
1,253,689,183,239,455,343,380,219,389,350,...,335,363,181,216,270,518,230,279,161,156
2,251,481,177,218,397,297,339,197,348,288,...,309,339,146,184,267,344,208,236,167,150
3,255,424,203,222,378,303,349,202,357,368,...,313,372,152,236,268,417,212,272,166,155
4,252,355,180,230,368,278,298,188,346,365,...,266,346,149,210,268,421,208,243,156,148
5,230,357,258,203,414,270,418,156,320,265,...,305,256,151,222,287,349,185,284,165,180


# 임의 가구 구성 셋팅

In [6]:
import json

In [7]:
# _m = m.sample(n=50, axis=1)
# _m = m[m.columns[:50]]

_m = m.copy()

# 초기 정보 셋팅
- 월, 공용부 차지 percentage

In [11]:
with open("test_data.json", "r") as f:
    test_data_info = json.load(f, encoding='utf8')

PUBLIC_PERCENTAGE = test_data_info['공용부 percentage']
_month = test_data_info['월']
h_cols = test_data_info['가구구성']

In [17]:
_m = _m[h_cols]
_m[_m.columns[:10]].head()

,아파트1-104-1208,아파트1-103-606,아파트1-104-705,아파트1-101-303,아파트3-101-603,아파트3-102-305,아파트1-103-1706,아파트2-2-1301,아파트3-103-707,아파트3-103-2018
month,,,,,,,,,,
1,40,415,290,376,258,223,381,290,262,147
2,135,399,263,335,219,201,330,236,204,132
3,169,421,281,349,246,225,368,268,221,160
4,127,401,257,343,224,209,381,258,212,165
5,7,398,84,356,196,204,336,141,204,216


In [18]:
month_df = pd.DataFrame(_m.loc[_month])
month_df.reset_index(inplace=True)

month_df.columns = ['name', 'usage (kWh)']

households_kWh = month_df['usage (kWh)'].sum()
households_count = len(month_df)
APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))

APT_mean = round(APT / households_count)
households_mean = round(month_df['usage (kWh)'].mean())

display(Markdown(
    "**설정 월은 {}월 입니다.**<br/>".format(_month) +
    "• 가구 수 : {}가구<br/>".format(households_count) +
    "• APT 총 사용량 : {}kWh (평균사용량:{}kWh)<br/>".format(format(APT,","),
                                                   APT_mean) +
    "• 세대부 총 사용량 : {}kWh (평균사용량:{}kWh)".format(format(households_kWh, ","),
                                              households_mean)
))

month_df.head()

**설정 월은 5월 입니다.**<br/>• 가구 수 : 396가구<br/>• APT 총 사용량 : 90,279kWh (평균사용량:228kWh)<br/>• 세대부 총 사용량 : 72,223kWh (평균사용량:182kWh)

,name,usage (kWh)
0,아파트1-104-1208,7
1,아파트1-103-606,398
2,아파트1-104-705,84
3,아파트1-101-303,356
4,아파트3-101-603,196


# 청구서 셋팅

In [20]:
calc = crs.models.ManagementOffice(
        month=_month,
        households=month_df,
        APT=APT,
        contract="단일계약"
    )
fee = calc.fee
fee

[[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [401, inf, 6060, 210.6]]

# 가구 분리

In [310]:
print()
display(Markdown("{}kWh 부터 {}kWh 까지 최저요금 적용 구간을 탐색합니다.".format(1, 200)))

for min_kwh in range(1, 200):
    hh = crs.models.Household(
        name="min test",
        kwh=min_kwh,
        contract=fee,
        contract_name="단일계약"
    )
    
    if hh.elec_bill > 1000:
        break
        
display(Markdown("### {}kWh 미만의 사용량은 최저요금 적용을 받습니다.".format(min_kwh)))

print()



1kWh 부터 200kWh 까지 최저요금 적용 구간을 탐색합니다.

### 37kWh 미만의 사용량은 최저요금 적용을 받습니다.

In [315]:
_month_df = month_df.copy()

In [333]:
min_month_df = _month_df[_month_df['usage (kWh)'] < min_kwh]
month_df = _month_df[_month_df['usage (kWh)'] >= min_kwh]

households_kWh = month_df['usage (kWh)'].sum()
# households_mean = round(month_df['usage (kWh)'].mean())

month_df.head()

,name,usage (kWh)
1,아파트1-103-606,398
2,아파트1-104-705,84
3,아파트1-101-303,356
4,아파트3-101-603,196
5,아파트3-102-305,204


In [334]:
min_month_df.head()

,name,usage (kWh)
0,아파트1-104-1208,7
10,아파트2-4-509,0
17,아파트2-2-807,8
21,아파트2-2-502,14
26,아파트2-4-1008,22


# 기본요금 예측

## 누진단계 셋팅

In [335]:
nugin_steps = [_[0] for _ in fee]
nugin_basic = [_[2] for _ in fee]

In [336]:
nugin_steps

[0, 201, 401]

## 누진단계 분할

In [337]:
nugin_dist = pd.DataFrame(columns=['가구 수'])

for step, _f in enumerate(fee):
    now_kWh = _f[0]
    if step < 2:
        next_kWh = fee[step + 1][0]
        count = len(month_df[(month_df['usage (kWh)'] >= now_kWh) &\
                             (month_df['usage (kWh)'] < next_kWh)])
        nugin_dist.loc['{} 단계'.format(step + 1)] = [count]
                    
    else:
        count = len(month_df[month_df['usage (kWh)'] >= now_kWh])
        nugin_dist.loc['{} 단계'.format(step + 1)] = [count]


nugin_dist.head()

,가구 수
1 단계,104
2 단계,187
3 단계,12


## 수거액 산정

In [338]:
collect_basic = list()

for step,count in enumerate(nugin_dist['가구 수'].values):
    basic = fee[step][2]
    
    collect_basic.append(basic * count)
    
nugin_dist['수거 금액'] = collect_basic
nugin_dist.loc['토탈'] = [nugin_dist['가구 수'].sum(), nugin_dist['수거 금액'].sum()]

display(Markdown("**세대부의 기본요금 수거 정보는 아래와 같습니다.**"))
# display(Markdown("**참고 요금표 : {}**".format(fee)))
for idx, (name, row) in enumerate(nugin_dist.iterrows()):
    if name != "토탈":
        display(Markdown(
            "• {} : {}원 * {}가구 = {}원<br/>".format(name, 
                                            format(fee[int(idx)][2], ","),
                                           row['가구 수'],
                                            format(row['수거 금액'], ","))
        ))
    else:
        _ex = reduce(lambda acc,cur: acc + "{} + ".format(format(cur, ",")), 
                    nugin_dist['수거 금액'].values[:3], "")
        display(Markdown(
            "##### 토탈 : {} = {}원".format(_ex, format(row['수거 금액'], ","))
        ))
    

nugin_dist.head()

**세대부의 기본요금 수거 정보는 아래와 같습니다.**

• 1 단계 : 730원 * 104가구 = 75,920원<br/>

• 2 단계 : 1,260원 * 187가구 = 235,620원<br/>

• 3 단계 : 6,060원 * 12가구 = 72,720원<br/>

##### 토탈 : 75,920 + 235,620 + 72,720 +  = 384,260원

,가구 수,수거 금액
1 단계,104,75920
2 단계,187,235620
3 단계,12,72720
토탈,303,384260


## APT 누진 단계 확인 및 기본요금 확인

In [339]:
display(Markdown("##### 현재 APT 평균사용량은 {}kWh 입니다.".format(APT_mean)))
print("요금표 {}를 참고하면,".format(fee))

for step, _f in enumerate(fee):
    if APT_mean < _f[1]:
        apt_step = step
        apt_basic = _f[2] * households_count
        break
        
display(Markdown("**현재 APT 평균사용량의 누진단계는 {}단계 입니다.**".format(apt_step + 1)))
display(Markdown("{}원의 기본요금이 적용되어, **아파트 전체 기본요금은 {} * {}가구 수 = {}원 입니다.**".format(
    format(fee[apt_step][2], ","),
    format(fee[apt_step][2], ","),
    households_count,
    format(apt_basic, ",")
)))

##### 현재 APT 평균사용량은 228kWh 입니다.

요금표 [[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [401, inf, 6060, 210.6]]를 참고하면,


**현재 APT 평균사용량의 누진단계는 2단계 입니다.**

1,260원의 기본요금이 적용되어, **아파트 전체 기본요금은 1,260 * 396가구 수 = 498,960원 입니다.**

## 공용부 기본요금 산출
- APT 기본요금 - 세대부 수거액

In [340]:
households_basic = nugin_dist.loc['토탈']['수거 금액'].sum()
public_basic = apt_basic - households_basic

display(Markdown("• APT 기본요금 : {}원".format(format(apt_basic, ","))))
display(Markdown("• 세대부 기본요금 : {}원".format(format(households_basic, ","))))
display(Markdown("#### 공용부의 기본요금은 {}원(APT 기본요금) - {}원(세대부 기본요금) = {}원 입니다.".format(
    format(apt_basic, ","),
    format(households_basic, ","),
    format(public_basic, ","))))

• APT 기본요금 : 498,960원

• 세대부 기본요금 : 384,260원

#### 공용부의 기본요금은 498,960원(APT 기본요금) - 384,260원(세대부 기본요금) = 114,700원 입니다.

# 전력량 요금 예측

## 누진단계별 수거해야 하는 사용량

In [341]:
col_list = list()
_usage = households_mean
for step, _f in enumerate(fee):
    collect_rate = _f[1] - _f[0]

    ## 2단계일 경우
    if step == 1:
        collect_rate += 1

    if households_mean < _f[1]:
        if (step == 0) or (step == 1):
            _col = _usage % collect_rate

        else:
            _col = _usage
    else:
        _col = collect_rate

    if _usage == 0:
        col_list.append(0)
    else:
        col_list.append(_col * households_count)
#         print(_col)
        _usage -= _col

apt_col = pd.DataFrame(col_list, columns=['수거 필요 사용량'])
apt_col.index = pd.Series(range(1, len(col_list) + 1), name="누진 단계")

display(Markdown("**현재 세대부 평균사용량은 {}kWh** 이며, 단계별로 아래와 같이 수거되어야 합니다.".format(
    format(households_mean, ",")
)))
apt_col

**현재 세대부 평균사용량은 182kWh** 이며, 단계별로 아래와 같이 수거되어야 합니다.

,수거 필요 사용량
누진 단계,
1,72072
2,0
3,0


## 가구별 누진 단계 수거 사용량

In [342]:
# nugin elec rate collect
nerc = pd.DataFrame(columns=['가구명', '누진단계', '수거 사용량'])

for idx, row in month_df.iterrows():
    name, usage = row
    
    col_list = list()
    _usage = usage
    for step, _f in enumerate(fee):
        collect_rate = _f[1] - _f[0]
            
        ## 2단계일 경우
        if step == 1:
            collect_rate += 1

        if usage < _f[1]:
            if (step == 0) or (step == 1):
                _col = _usage % collect_rate
                
            else:
                _col = _usage
        else:
            _col = collect_rate
            
        if _usage == 0:
            break
        else:
            col_list.append(_col)
            _usage -= _col
            
    for step,col in enumerate(col_list):
        nerc = nerc.append({
            "가구명": name,
            "누진단계": step + 1,
            "수거 사용량": col
        }, ignore_index=True)
        
nerc = nerc.pivot_table(index=['누진단계'], values=['수거 사용량'], aggfunc='sum')
nerc

,수거 사용량
누진단계,
1,53983
2,17167
3,415


### 가구 수거액 확인

In [343]:
col_pd = pd.concat([apt_col, nerc], axis=1).fillna(0).astype("int")
col_pd['오차'] = col_pd['수거 필요 사용량'] - col_pd['수거 사용량']

col_pd.index = col_pd.index.map(lambda x: "{} 단계".format(x))

col_pd

,수거 필요 사용량,수거 사용량,오차
1 단계,72072,53983,18089
2 단계,0,17167,-17167
3 단계,0,415,-415


## 공용부 요금 시작 지점 확인

In [344]:
public_start = 0
h_elec_rate_mem = list()

print()
for idx, (step, row) in enumerate(col_pd.iterrows()):
    _f = fee[idx]
    rate = row['오차']
    
    display(Markdown("• {} 단계 : {}원 * {}kWh = {}원".format(
        idx + 1,
        format(_f[3], ","),
        format(rate, ","),
        format((_f[3] * rate), ",")
    )))
    h_elec_rate_mem.append((_f[3] * rate))
    public_start += (_f[3] * rate)

print(reduce(lambda acc, cur: acc + " - {}원".format(abs(cur))  if cur < 0 else acc + " + {}원".format(abs(cur)),
        h_elec_rate_mem,
        "")[2:],"=","{}원".format(format(public_start, ",")))
public_start = round(public_start)



display(Markdown("#### 공용부 시작 전력량 요금은 {}원(4사 5입) 입니다.".format(format(public_start, ","))))

print()

• 1 단계 : 73.3원 * 18,089kWh = 1,325,923.7원

• 2 단계 : 142.3원 * -17,167kWh = -2,442,864.1원

• 3 단계 : 210.6원 * -415kWh = -87,399.0원

 1325923.7원 - 2442864.1원 - 87399.0원 = -1,204,339.4000000001원


#### 공용부 시작 전력량 요금은 -1,204,339원(4사 5입) 입니다.

## 공용부 시작 누진 단계 확인

In [345]:
col_list = list()
_usage = APT_mean
for step, _f in enumerate(fee):
    collect_rate = _f[1] - _f[0]

    ## 2단계일 경우
    if step == 1:
        collect_rate += 1

    if APT_mean < _f[1]:
        if (step == 0) or (step == 1):
            _col = _usage % collect_rate
        else:
            _col = _usage
    else:
        _col = collect_rate

    if _usage == 0:
        break
    else:
        col_list.append(_col)
        _usage -= _col
        
_usage = households_mean
for idx,col in enumerate(col_list):
    if _usage < col:
        col_list[idx] -= _usage
        _usage -= _usage
    else:
        _usage -= col_list[idx]
        col_list[idx] = 0
        
    if _usage == 0:
        break
        
elec_rate_start = np.where(np.array(col_list) != 0)[0][0]
display(Markdown("#### 공용부 전력량 요금은 {}단계부터 시작합니다. 해당 단계의 나머지 사용량은 {}kWh입니다.".format(
    elec_rate_start + 1,
    col_list[elec_rate_start]
)))

for idx,col in enumerate(col_list):
    display(Markdown("• {} 단계 : {}kWh".format(
        elec_rate_start + 1 + idx,
        col
    )))

#### 공용부 전력량 요금은 1단계부터 시작합니다. 해당 단계의 나머지 사용량은 18kWh입니다.

• 1 단계 : 18kWh

• 2 단계 : 28kWh

## 공용부 전력량 요금 계산 및 산출

In [346]:
print()

public_elec_rate = 0
memories = list()

for step,col in enumerate(col_list):
    _f = fee[step]
    
    display(Markdown("• {} 단계 : {}원 * {}kWh * {}가구 = {}원".format(
        step + 1,
        format(_f[3], ","),
        format(col, ","),
        format(households_count, ","),
        format((_f[3] * col * households_count), ",")
    )))
    
    memories.append(_f[3] * col * households_count)
    public_elec_rate += _f[3] * col * households_count
    
display(Markdown("\t{} {} {}".format(reduce(lambda acc, cur: acc + " - {}원".format(abs(cur))  if cur < 0 else acc + " + {}원".format(abs(cur)),
        memories,
        "")[2:],"=","{}원(4사 5입)".format(format(round(public_elec_rate), ",")))))
display(Markdown("\t{}원(공용부 전력량요금) + {}원(공용부 시작 전력량요금)".format(
    format(round(public_elec_rate), ","),
    format(public_start, ",")
)))
    
public_elec_rate = round(public_elec_rate) + public_start

display(Markdown("#### 공용부 전력량 요금은 {}원 입니다.".format(format(public_elec_rate, ","))))
print()

• 1 단계 : 73.3원 * 18kWh * 396가구 = 522,482.39999999997원

• 2 단계 : 142.3원 * 28kWh * 396가구 = 1,577,822.4000000001원

	 522482.39999999997원 + 1577822.4000000001원 = 2,100,305원(4사 5입)

	2,100,305원(공용부 전력량요금) + -1,204,339원(공용부 시작 전력량요금)

#### 공용부 전력량 요금은 895,966원 입니다.

# 기후환경요금, 연료비조정액 예측

In [347]:
public_env = round((APT - households_kWh) * 5.3)
public_fuel = round((APT - households_kWh) * 3)

print()
display(Markdown("공용부 기후환경요금 = ({}kWh(APT 총 사용량) - {}kWh(세대부 총 사용량)) * 5.3원 = **{}원**".format(
    format(APT, ","),
    format(households_kWh, ","),
    format(public_env, ",")
)))
display(Markdown("공용부 연료비조정액 = ({}kWh(APT 총 사용량) - {}kWh(세대부 총 사용량)) * 3원 = **{}원**".format(
    format(APT, ","),
    format(households_kWh, ","),
    format(public_fuel, ",")
)))
print()

공용부 기후환경요금 = (90,279kWh(APT 총 사용량) - 71,565kWh(세대부 총 사용량)) * 5.3원 = **99,184원**

공용부 연료비조정액 = (90,279kWh(APT 총 사용량) - 71,565kWh(세대부 총 사용량)) * 3원 = **56,142원**

# 전기요금계 예측

In [364]:
print()
public_elec_bill = public_basic + public_elec_rate + public_env - public_fuel
display(Markdown("필수사용량보장공제 대상은 **{}가구** 입니다.".format(cnt)))
display(Markdown("필수사용량보장공제 = {}원 * {}가구 = **{}원**".format(format(2500, ","),
                                                       cnt,
                                                      format(households_guarantees, ","))))
display(Markdown("{}원(기본요금) + {}원(전력량요금) + {}원(기후환경요금) - {}원(연료비조정액)".format(
    format(public_basic, ","),
    format(public_elec_rate, ","),
    format(public_env, ","),
    format(public_fuel, ",")
)))
display(Markdown("#### 공용부 전기요금계는 {}원 입니다.".format(format(public_elec_bill, ","))))
print()

필수사용량보장공제 대상은 **197가구** 입니다.

필수사용량보장공제 = 2,500원 * 197가구 = **260,000원**

114,700원(기본요금) + 895,966원(전력량요금) + 99,184원(기후환경요금) - 56,142원(연료비조정액)

#### 공용부 전기요금계는 1,053,708원 입니다.

### 필수사용량 보장공제

In [365]:
print()
cnt = 0
for _ in calc.households:
    if _.guarantee != 0:
        cnt+=1
        
households_guarantees = sum([2500 if _ <= 200 else 0 for _ in month_df['usage (kWh)'].values])


display(Markdown("필수사용량보장공제 대상은 **{}가구** 입니다.".format(cnt)))
display(Markdown("필수사용량보장공제 = {}원 * {}가구 = **{}원**".format(format(2500, ","),
                                                       cnt,
                                                      format(households_guarantees, ","))))
display(Markdown("{}원(전기요금계) + {}원(필수사용량보장공제)".format(
    format(public_elec_bill, ","),
    format(households_guarantees, ",")
)))

public_elec_bill += households_guarantees
display(Markdown("#### 공용부 전기요금계는 {}원 입니다.".format(format(public_elec_bill, ","))))

print()

필수사용량보장공제 대상은 **197가구** 입니다.

필수사용량보장공제 = 2,500원 * 197가구 = **260,000원**

1,053,708원(전기요금계) + 260,000원(필수사용량보장공제)

#### 공용부 전기요금계는 1,313,708원 입니다.

### 월 최저요금 총 합

In [366]:
min_rate = 1000 * len(min_month_df)

print()
display(Markdown("월 최저요금 적용가구는 **{}가구** 입니다.".format(len(min_month_df))))
display(Markdown("월 최저요금 총 합 = {}원 * {}가구 = **{}원**".format(format(100, ","),
                                                       len(min_month_df),
                                                      format(min_rate, ","))))
display(Markdown("{}원(전기요금계) + {}원(월 최저요금 총 합)".format(
    format(public_elec_bill, ","),
    format(min_rate, ",")
)))

public_elec_bill -= min_rate
display(Markdown("#### 공용부 전기요금계는 {}원 입니다.".format(format(public_elec_bill, ","))))
print()

월 최저요금 적용가구는 **93가구** 입니다.

월 최저요금 총 합 = 100원 * 93가구 = **93,000원**

1,313,708원(전기요금계) + 93,000원(월 최저요금 총 합)

#### 공용부 전기요금계는 1,220,708원 입니다.

In [367]:
public_vat = round(public_elec_bill * 0.1)
public_fund = round(public_elec_bill * 0.037)


print()
display(Markdown("공용부 부가가치세 = {}원 * 0.1 = **{}원**".format(
    format(public_elec_bill, ","),
    format(public_vat, ",")
)))
display(Markdown("공용부 전력산업기반기금 = {}원 * 0.037 = **{}원**".format(
    format(public_elec_bill, ","),
    format(public_fund, ",")
)))
print()

공용부 부가가치세 = 1,220,708원 * 0.1 = **122,071원**

공용부 전력산업기반기금 = 1,220,708원 * 0.037 = **45,166원**

In [368]:
public_bill = public_elec_bill + public_vat + public_fund

print()
display(Markdown("{}원 + {}원 + {}원".format(
    format(public_elec_bill,","),
    format(public_vat,","),
    format(public_fund,",")
)))
display(Markdown("공용부 예측 최종청구금액 : **{}원**".format(format(public_bill, ","))))
display(Markdown("공용부 실제 최종청구금액 : **{}원**".format(format(calc.public_bill, ","))))
display(Markdown("오차 : **{}원**".format(format(abs(public_bill - calc.public_bill), ","))))
print()

1,220,708원 + 122,071원 + 45,166원

공용부 예측 최종청구금액 : **1,387,945원**

공용부 실제 최종청구금액 : **1,391,330원**

오차 : **3,385원**